## Imports

In [12]:
!pip install xmltodict
import os
import torch
import pandas as pd
import numpy as np
import torchvision
import torch.nn as nn
from torch.utils.data import Dataset, random_split, DataLoader
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.utils import make_grid
from torchvision.datasets import VisionDataset
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
from collections import OrderedDict
import xmltodict
%matplotlib inline

## Creating a custom dataset that will hold the labeled data

In [13]:
class CustomDataset(VisionDataset):
    def __init__(self, root_dir, transform=None, target_transform=None):
        super().__init__(root_dir, transform=transform, target_transform=target_transform)

        self.image_paths = []
        self.targets = []

        # Iterate over all XML files in root_dir
        for xml_file in os.listdir(root_dir):
            if not xml_file.endswith('.xml'):
                continue

            with open(os.path.join(root_dir, xml_file)) as f:
                xml_data = xmltodict.parse(f.read())

            # Extract label and image path from XML file
            label = xml_data['annotation']['object']['name']
            image_path = os.path.join(root_dir, xml_data['annotation']['filename'])

            self.targets.append(label)
            self.image_paths.append(image_path)

    def __getitem__(self, index):
        # Load image and convert to tensor
        image = Image.open(self.image_paths[index])
        image = self.transform(image)

        # Convert label to tensor
        target = self.targets[index]
        target = torch.tensor(target)

        return image, target

    def __len__(self):
        return len(self.image_paths)


In [14]:
train_dataset = CustomDataset(r'C:\Users\User\Desktop\ObjectDetection\ptod\Image Collection\workspace\images\train')
test_dataset = CustomDataset(r'C:\Users\User\Desktop\ObjectDetection\ptod\Image Collection\workspace\images\test')


In [15]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=False)
